In [9]:
#### Import Libraries ####
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt

In [10]:
#### Region of Interest Selection ####
def ROI(image):
    ## create mask
    mask = np.zeros_like(image)

    if len(image.shape) > 2:
        ignore_mask_color = (255, ) * image.shape[2]
    else:               ##for the greyscale images
        ignore_mask_color = 255

    ## define the polygon to focus on the road
    rows, cols = image.shape[:2]

    bottom_left  = [cols * 0.1, rows * 0.95]
    top_left     = [cols * 0.4, rows * 0.6]
    bottom_right = [cols * 0.9, rows * 0.95]
    top_right    = [cols * 0.7, rows * 0.6]

    vertices = np.array([[bottom_left, top_left, top_right, bottom_right]], np.int32)

    # Fill the polygon with the white color to get the mask
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    # cv2.imshow('Mask Shape',mask)
    # Perform Bitwise AND operation to get the ROI
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

In [19]:
#### Hough Transform Function ####
def Hough_Transform(image):
    rho = 1
    theta = np.pi/180
    threshold = 20
    minLineLength = 20
    maxLineGap = 500
    return cv2.HoughLinesP(image, rho = rho, theta = theta, threshold = threshold, minLineLength = minLineLength, maxLineGap = maxLineGap)

In [12]:
def slope_intercept_avg(lines):
    # Lines: Points calculated from Hough Transform x1,y1,x2,y2
    slope_intercept_left = [] # store list of touple(slope, intercept of left lane)
    length_left = [] # list of lengths of left lane
    slope_intercept_right = [] # store list of touple(slope, intercept of right lane)
    length_right = [] # list of lengths of right lane

    for line in lines:
        for x1, y1, x2, y2  in line:
            # check if the line is vertical
            if x1==x2:
                continue
            # slope of the line
            slope = (y2 - y1) / (x2 - x1)
            # intercept of the line
            intercept = y1 - (slope * x1)
            # length of the line
            length = np.sqrt(((y2 - y1) ** 2)+ ((x2 - x1) ** 2))

            ## Observe that the slope of the left line is negative and right lane is positive 
            ## The reason being that the origin(0,0) for images is located at top left with
            ## positive x-axis extending right and positive y axis extending downwards

            if slope < 0:
                slope_intercept_left.append((slope, intercept))
                length_left.append(length)
            else:
                slope_intercept_right.append((slope, intercept))
                length_right.append(length)

            ## np.dot() essentially combines the information from multiple detected lines into a single representative line, 
            ## giving more weight to lines with higher confidence or longer lengths.

            left_lane = np.dot(length_left, slope_intercept_left) / np.sum(length_left) if len(length_left) > 0 else None
            right_lane = np.dot(length_right, slope_intercept_right) / np.sum(length_right) if len(length_right) > 0 else None

            return left_lane, right_lane

In [13]:
def pixel_points(y1, y2, line):
    
    # Converts the slope and intercept of each line into pixel points.
    #     Parameters:
    #         y1: y-value of the line's starting point.
    #         y2: y-value of the line's end point.
    #         line: The slope and intercept of the line.
    
    if line is None:
        return None

    slope, intercept = line
    x1 = int((y1 - intercept)/(slope))
    x2 = int((y2 - intercept)/(slope))
    y1 = int(y1)
    y2 = int(y2)

    return ((x1, y1), (x2, y2))


In [14]:
def lane_lines(image, lines):
    
    # Create full lenght lines from pixel points.
    #     Parameters:
    #         image: The input test image.
    #         lines: The output lines from Hough Transform.
    
    left_lane, right_lane = slope_intercept_avg(lines=lines)
    y1 = image.shape[0]     # height of the image
    y2 = y1 * 0.5
    left_line = pixel_points(y1, y2, left_lane)     ## Create left line to draw on image
    right_line = pixel_points(y1, y2, right_lane)   ## Create right line to draw on image
    return left_line, right_line 

In [15]:
def draw_lane_lines(image, lines, color = [0,0,255], thickness = 10):
    # Draw lines onto the input image.
    #     Parameters:
    #         image: The input test image (video frame in our case).
    #         lines: The output lines from Hough Transform.
    #         color (Default = red): Line color.
    #         thickness (Default = 12): Line thickness. 

    line_mask = np.zeros_like(image)
    for line in lines:
        if line is not None:
            cv2.line(line_mask, *line, color, thickness)
    return cv2.addWeighted(image, 1.0, line_mask, 1.0, 0.0)
     

In [18]:
capture = cv2.VideoCapture("Dataset/test2.mp4")

if capture.isOpened() == False:
    print('ERROR FILE NOT FOUND OR WRONG CODEC')

while capture.isOpened():

    ret, frame = capture.read()
    # print(type(frame))

    if ret == True:
        
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        kernel_size = 5
        blur = cv2.GaussianBlur(gray_frame, (kernel_size, kernel_size), 0)

        thres_low, thres_high = 100, 200

        edges = cv2.Canny(blur, thres_low, thres_high)
        region = ROI(edges)

        hough_points = Hough_Transform(region)
        
        lines_on_image = draw_lane_lines(frame, lane_lines(frame, hough_points))
        
        

        cv2.imshow('Frame', lines_on_image)
       
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    
    else:
        break

capture.release()
cv2.destroyAllWindows()